In [12]:
import torch
from torch import nn
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

# Prepare dataset
instead of predicting sequence it predicts pairwise interaction matrix

In [18]:
class RNADataset(Dataset):
    def __init__(self, path: str):
        """path: path to .csv file with sequences and structures"""
        self.data = pd.read_csv(path)

    def __len__(self):
        return len(self.data)
    
    def create_interaction_matrix(structure: str) -> torch.tensor:
        stack = [[], [], []]
        matrix = torch.zeros(len(structure), len(structure), dtype=torch.bfloat16)

        for i in range(len(structure)):
            match structure[i]:
                case "(":
                    stack[0].append(i)
                case ")":
                    matrix[stack[0].pop(), i] = 1
                case "[":
                    stack[1].append(i)
                case "]":
                    matrix[stack[1].pop(), i] = 1
                case "{":
                    stack[2].append(i)
                case "}":
                    matrix[stack[2].pop(), i] = 1
                case ".":
                    continue
        return matrix
    
    def tokenize(sequence: str) -> torch.tensor:
        VOCABULARY = {
            "A": torch.int8(0),
            "U": torch.int8(1),
            "C": torch.int8(2),
            "G": torch.int8(3),
        }

        return torch.tensor([VOCABULARY[i] for i in sequence])
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        sequence = row["sequence"]
        tokenized_sequence = self.tokenize(sequence)
        structure = row["structure"]
        matrix = self.create_interaction_matrix(structure)
        
        return tokenized_sequence, matrix